# BIG DATA ASSIGNMENT WEEK 09
## Colaborative Filtering
- Rafi Akbar Rafsanjani
- 05111942000004

## Install & Initialization

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 14.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=badccb944eb4b84c54d2eda2bd2552994f216141a1134cb35cd2411d6a111966
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

In [15]:
# SparkSession Initialization
spark = SparkSession.builder \
    .master("local") \
    .appName("MyApp") \
    .getOrCreate()

In [16]:
# Read data from a text file and separate elements of each line
lines = spark.read.text("/content/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))

In [17]:
# Convert data into a DataFrame with userId, movieId, rating, and timestamp columns
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))

# Split the data into training (80%) and testing (20%) sets
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

## Build Recomendation model using ALS

In [18]:
# Initialize the parameters to be tried
max_iters = [5, 10, 20]
reg_params = [0.1, 0.5, 1.0]

# Dictionary to store RMSE results
results = {}

In [19]:
# Loop for every combination of maxIter and regParam
for max_iter in max_iters:
    for reg_param in reg_params:
        # Build the recommendation model using ALS on the training data
        # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
        als = ALS(maxIter=max_iter, regParam=reg_param, userCol="userId", itemCol="movieId", ratingCol="rating",
                  coldStartStrategy="drop")
        model = als.fit(training)

        # Evaluate the model by computing the RMSE on the test data
        predictions = model.transform(test)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)

        # Save the RMSE result in the dictionary
        results[(max_iter, reg_param)] = rmse
        print(f"Root-mean-square error for maxIter={max_iter}, regParam={reg_param} = {rmse}")

Root-mean-square error for maxIter=5, regParam=0.1 = 1.0057921046499112
Root-mean-square error for maxIter=5, regParam=0.5 = 1.2257729289203048
Root-mean-square error for maxIter=5, regParam=1.0 = 1.5271928534289692
Root-mean-square error for maxIter=10, regParam=0.1 = 0.9351503922563846
Root-mean-square error for maxIter=10, regParam=0.5 = 1.2201469771778586
Root-mean-square error for maxIter=10, regParam=1.0 = 1.5271839911362288
Root-mean-square error for maxIter=20, regParam=0.1 = 0.9074862498103773
Root-mean-square error for maxIter=20, regParam=0.5 = 1.2198238887009718
Root-mean-square error for maxIter=20, regParam=1.0 = 1.5271839983811029


In [20]:
# Find the hyperparameter combination with the lowest RMSE
best_params = min(results, key=results.get)
best_rmse = results[best_params]
print(f"\nBest hyperparameters: maxIter={best_params[0]}, regParam={best_params[1]} with RMSE={best_rmse}")


Best hyperparameters: maxIter=20, regParam=0.1 with RMSE=0.9074862498103773


## Generate Movie Recomendation

In [21]:
# Train the model with the best hyperparameters
best_als = ALS(maxIter=best_params[0], regParam=best_params[1], userCol="userId", itemCol="movieId", ratingCol="rating",
               coldStartStrategy="drop")
best_model = best_als.fit(training)

## Print Result and Show Ouput

In [22]:
# Generate top 10 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 3.9311185},...|
|    10|[{92, 3.5439057},...|
|     0|[{92, 3.2136796},...|
|     1|[{62, 3.129572}, ...|
|    21|[{29, 4.1986437},...|
|    11|[{30, 4.69603}, {...|
|    12|[{46, 4.713656}, ...|
|    22|[{51, 4.582819}, ...|
|     2|[{93, 4.648805}, ...|
|    13|[{93, 3.212728}, ...|
|     3|[{51, 3.9556918},...|
|    23|[{32, 4.7867374},...|
|     4|[{29, 3.4560595},...|
|    24|[{30, 4.4359}, {6...|
|    14|[{52, 4.2949166},...|
|     5|[{55, 3.5791166},...|
|    15|[{46, 4.0466633},...|
|    25|[{47, 3.0355759},...|
|    26|[{22, 4.6835976},...|
|     6|[{25, 3.9362824},...|
+------+--------------------+
only showing top 20 rows



In [23]:
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 3.869947}, ...|
|     40|[{2, 3.4636762}, ...|
|     10|[{12, 3.4265466},...|
|     50|[{23, 3.8660583},...|
|     80|[{3, 3.3820553}, ...|
|     70|[{21, 3.0555878},...|
|     60|[{3, 2.5913453}, ...|
|     90|[{24, 4.183577}, ...|
|     30|[{11, 4.69603}, {...|
|      0|[{28, 2.6888108},...|
|     31|[{12, 2.8959033},...|
|     81|[{28, 4.017785}, ...|
|     91|[{12, 2.9185693},...|
|      1|[{15, 2.9199452},...|
|     41|[{21, 3.2116048},...|
|     61|[{6, 2.1022265}, ...|
|     51|[{26, 4.6338367},...|
|     21|[{26, 2.7642434},...|
|     11|[{18, 3.362848}, ...|
|     71|[{25, 2.892922}, ...|
+-------+--------------------+
only showing top 20 rows



In [24]:
# Generate top 10 movie recommendations for a specific set of users
users = ratings.select(best_als.getUserCol()).distinct().limit(3)
userSubsetRecs = best_model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{22, 4.6835976},...|
|    19|[{22, 3.1230206},...|
|    29|[{46, 4.2881436},...|
+------+--------------------+



In [25]:
# Generate top 10 user recommendations for a specific set of movies
movies = ratings.select(best_als.getItemCol()).distinct().limit(3)
movieSubSetRecs = best_model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     65|[{23, 4.043209}, ...|
|     26|[{28, 2.5480118},...|
|     29|[{8, 4.4113417}, ...|
+-------+--------------------+



## Summary

To summarize the process, the team performed data loading and preprocessing by splitting the MovieLens dataset into training and test sets. They then conducted hyperparameter tuning by testing various combinations of maxIter and regParam values and recorded the results in an RMSE dictionary. The team selected the best hyperparameters based on the lowest RMSE and displayed the results. Lastly, they trained the model using the best hyperparameters and produced recommendations for all users, movies, and specific subsets. The recommendations included the top 10 suggestions for each user, movie, and selected subset.



